<a href="https://colab.research.google.com/github/CS20M038/CS6910-assignment_3/blob/main/DL_Assignment_3_Arjun_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setting Up the Notebook
This section involves the importing of important librabries,setting wandb,checking for GPU and mounting onto the drive.

In [1]:
#import all the libraries that would be used in this notebook
import csv
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers.embeddings import Embedding
from tensorflow.keras.layers import LSTM,Dense
from keras.models import Model
from keras.utils.vis_utils import plot_model

In [2]:
!pip install wandb -qqq
import wandb
wandb.login()

     |████████████████████████████████| 1.8MB 8.0MB/s 
     |████████████████████████████████| 133kB 56.4MB/s 
     |████████████████████████████████| 174kB 43.6MB/s 
     |████████████████████████████████| 102kB 12.8MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
# to mount to the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
else:
  print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Loading Data and Preprocessing 

In [5]:
#make sure to upload the Dakshina Dataset in your drive and name the folder as Dakshina_Dataset
dakshina_dataset_hindi = '/content/drive/MyDrive/Dakshina_Dataset/hi/lexicons'

In [6]:
#all the dataset paths
dev_dataset_path = os.path.join(dakshina_dataset_hindi,"hi.translit.sampled.dev.tsv")
train_dataset_path = os.path.join(dakshina_dataset_hindi,"hi.translit.sampled.train.tsv")
test_dataset_path = os.path.join(dakshina_dataset_hindi,"hi.translit.sampled.test.tsv")

In [7]:
#function to load the dataset from the dataset path
def load_dataset(path,test_dataset = False):
    dataset = open(path)
    read_dataset = csv.reader(dataset,delimiter = '\t')
    x = []
    y = []
    for row in read_dataset:
        x.append(row[1])
        if test_dataset:
            y.append(row[0])
        else:
            y.append("\t"+row[0]+'\n')

    return np.array(x),np.array(y)


In [89]:
x_raw_train,y_raw_train = load_dataset(train_dataset_path,False)
x_raw_val,y_raw_val = load_dataset(dev_dataset_path,True)
x_raw_test,y_raw_test = load_dataset(test_dataset_path,True)

In [9]:
english_alphabets = 'abcdefghijklmnopqrstuvwxyz'
english_alpha2index = {"PAD": 0}
for index,alpha in enumerate(english_alphabets):
    english_alpha2index[alpha] = index + 1

hindi_alphabets = [chr(alpha) for alpha in range(2304, 2432)]
hindi_alphabets.append('\t')
hindi_alphabets.append('\n')

hindi_alpha2index = {"PAD" : 0}
for index,alpha in enumerate(hindi_alphabets):
    hindi_alpha2index[alpha] = index + 1

english_index2alpha = {0: "PAD"}
hindi_index2alpha = {0: "PAD"}


for index,alpha in enumerate(english_alphabets):
    english_index2alpha[index + 1] = alpha

for index,alpha in enumerate(hindi_alphabets):
    hindi_index2alpha[index + 1] = alpha


In [10]:
def get_integer_encode(word,alpha2index,max_length = 25):
    integer_encode = np.zeros((max_length,),dtype='int')
    for index,alpha in enumerate(word):
        integer_encode[index] = alpha2index[alpha]
    return integer_encode

def get_word_decode(integer_encoded,index2alpha):
    
    word = ""
    for integer in integer_encoded:
        word += index2alpha[integer]
    return word


def encode_docs(docs,alpha2index,max_length = 25):
    encoded_docs = np.zeros((docs.shape[0],max_length),dtype='int')
    for index,word in enumerate(docs):
        encoded_docs[index] = get_integer_encode(word,alpha2index)
    return encoded_docs



In [11]:
integer_encoded_x_train = encode_docs(x_raw_train,english_alpha2index)
integer_encoded_y_train = encode_docs(y_raw_train,hindi_alpha2index)

In [12]:
decoder_input_data = np.zeros((integer_encoded_y_train.shape[0],25,len(hindi_alpha2index)),dtype="float32")
decoder_output_data = np.zeros((integer_encoded_y_train.shape[0],25,len(hindi_alpha2index)),dtype="float32")

for i,integer_encoded_data in enumerate(integer_encoded_y_train):
    for t,integer in enumerate(integer_encoded_data):
        decoder_input_data[i, t, integer]= 1.0
        if t > 0:
            decoder_output_data[i, t - 1, integer] = 1.0

In [33]:
integer_encoded_x_val = encode_docs(x_raw_val,english_alpha2index)
integer_encoded_y_val = encode_docs(y_raw_val,hindi_alpha2index)

In [34]:
decoder_input_val = np.zeros((integer_encoded_y_val.shape[0],25,len(hindi_alpha2index)),dtype="float32")
decoder_output_val = np.zeros((integer_encoded_y_val.shape[0],25,len(hindi_alpha2index)),dtype="float32")

for i,integer_encoded_data in enumerate(integer_encoded_y_val):
    for t,integer in enumerate(integer_encoded_data):
        decoder_input_val[i, t, integer]= 1.0
        if t > 0:
            decoder_output_val[i, t - 1, integer] = 1.0

# The Generic RNN Class

In [26]:
class Customcallback(keras.callbacks.Callback):

    def __init__(self):
        
    def on_epoch_end(self,epoch,logs = None):
        print("in epoch",epoch)
        print(logs)

        

In [83]:
class RNN():

    def __init__(self,embedding_size,num_encoder_layers,num_decoder_layers,hidden_layer_size,cell_type,drop_out_ratio,in_char_size,out_char_size,input_len = 25):

        self.embedding_size = embedding_size
        self.num_encoder_layers = num_encoder_layers
        self.num_decoder_layers = num_decoder_layers
        self.hidden_layer_size = hidden_layer_size
        self.cell_type = cell_type
        self.drop_out_ratio = drop_out_ratio
        self.in_char_size = in_char_size
        self.out_char_size = out_char_size
        self.input_len = input_len

        self._build_rnn_network()

    def _build_rnn_network(self):

        # Embedding 
        embedding_inputs = keras.Input(shape=(None,))
        embedding_layer = Embedding(self.in_char_size, self.embedding_size , input_length=self.input_len)
        encoder_inputs = embedding_layer(embedding_inputs)

        #Encoder

        self.encoder_layers = []
        for _ in range(self.num_encoder_layers-1):
            encoder_layer = getattr(layers,self.cell_type)(self.hidden_layer_size, dropout=self.drop_out_ratio, return_sequences=True)
            encoder_inputs = encoder_layer(encoder_inputs)
            self.encoder_layers.append(encoder_layer)

        last_encoder_layer = getattr(layers,self.cell_type)(self.hidden_layer_size, dropout=self.drop_out_ratio, return_state=True)
        encoder_outputs,*encoder_states = last_encoder_layer(encoder_inputs)
        self.encoder_layers.append(last_encoder_layer)

        #Decoder
        initial_decoder_inputs = keras.Input(shape=(None, self.out_char_size))
        decoder_inputs = initial_decoder_inputs

        self.decoder_layers = []

        for _ in range(self.num_decoder_layers):

            decoder_layer = getattr(layers,self.cell_type)(self.hidden_layer_size, dropout=self.drop_out_ratio, return_sequences=True,return_state=True)
            decoder_inputs,*decoder_states = decoder_layer(decoder_inputs,initial_state=encoder_states)
            self.decoder_layers.append(decoder_layer)

        decoder_outputs = decoder_inputs
        decoder_dense = Dense(self.out_char_size, activation="softmax")
        decoder_outputs = decoder_dense(decoder_outputs)

        model = keras.Model([embedding_inputs, initial_decoder_inputs], decoder_outputs) 

        self.model = model
        self.embedding_inputs = embedding_inputs
        self.encoder_states = encoder_states
        self.decoder_inputs = initial_decoder_inputs
        self.decoder_dense = decoder_dense

    def compile(self,optimizer="rmsprop"):

        self.model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
        )

    def fit(self,encoder_input,decoder_input,decoder_output,batch_size = 64,epochs = 5):
        self.model.fit(
        [encoder_input, decoder_input],
        decoder_output,
        batch_size=batch_size,
        epochs=epochs,
        callbacks = [wandb.keras.WandbCallback()],
        validation_split=0.1
        )

    def build_inference_model(self):

        self.encoder_model = Model(self.embedding_inputs,self.encoder_states)

        decoder_state_input = []
        for i in range(len(self.encoder_states)) :
            new_state = keras.Input(shape=(self.hidden_layer_size,))
            decoder_state_input.append(new_state)

        initial_decoder_inputs = self.decoder_inputs

        decoder_inputs = initial_decoder_inputs
        for layer in self.decoder_layers :
            decoder_inputs, *decoder_states = layer(decoder_inputs,initial_state=decoder_state_input)

        decoder_outputs = decoder_inputs
        decoder_outputs = self.decoder_dense(decoder_outputs)

        self.decoder_model = Model(
            [initial_decoder_inputs] + decoder_state_input,
            [decoder_outputs] + decoder_states
            )


    def _decode_sequence(self,word):

        input_seq = get_integer_encode(word,english_alpha2index)
        input_seq = input_seq.reshape(1,25)
    
        states_value = self.encoder_model.predict(input_seq)

        target_seq = np.zeros((1,1, len(hindi_alpha2index)))
        target_seq[0,0, hindi_alpha2index["\t"]] = 1.0

        decoded_word = ""
        while True:
            output_tokens, *states = self.decoder_model.predict([target_seq] + [states_value])

            sampled_token_index = np.argmax(output_tokens[0, -1, :])

            sampled_char = hindi_index2alpha[sampled_token_index]
        
            if sampled_char == "\n" or len(decoded_word) > 25:
                break

            decoded_word += sampled_char

            target_seq = np.zeros((1, 1,len(hindi_alpha2index)))
            target_seq[0, 0, sampled_token_index] = 1.0

            states_value = [states[i] for i in range(len(states))]

        return decoded_word

    def evaluate(self,X_test,Y_test):
        correct = 0
        for english_word,hindi_word in zip(X_test,Y_test):
            predicted_hindi_word = self._decode_sequence(english_word)
            if predicted_hindi_word == hindi_word:
                correct += 1

        acc = (correct/X_test.shape[0])*100
        return acc


In [84]:
hyperparameters = {
    "embedding_size" : 125,
    "num_encoder_layers" : 1,
    "num_decoder_layers" : 1,
    "hidden_layer_size" : 256,
    "cell_type" : "LSTM",
    "drop_out_ratio": 0.2,
    "in_char_size": len(english_alpha2index),
    "out_char_size": len(hindi_alpha2index),
    "input_len": 25,
    }

net = RNN(**hyperparameters)


In [85]:
net.compile()
net.fit(integer_encoded_x_train, decoder_input_data,decoder_output_data,epochs = 25)

Epoch 1/25
691/691 [==============================] - 11s 12ms/step - loss: 1.1295 - accuracy: 0.7370 - val_loss: 0.7722 - val_accuracy: 0.7994
Epoch 2/25
691/691 [==============================] - 7s 11ms/step - loss: 0.7803 - accuracy: 0.7937 - val_loss: 0.6081 - val_accuracy: 0.8323
Epoch 3/25
691/691 [==============================] - 7s 11ms/step - loss: 0.6390 - accuracy: 0.8254 - val_loss: 0.4880 - val_accuracy: 0.8647
Epoch 4/25
691/691 [==============================] - 7s 11ms/step - loss: 0.5085 - accuracy: 0.8580 - val_loss: 0.3742 - val_accuracy: 0.8920
Epoch 5/25
691/691 [==============================] - 7s 11ms/step - loss: 0.4062 - accuracy: 0.8827 - val_loss: 0.3133 - val_accuracy: 0.9123
Epoch 6/25
691/691 [==============================] - 7s 11ms/step - loss: 0.3325 - accuracy: 0.9019 - val_loss: 0.2567 - val_accuracy: 0.9251
Epoch 7/25
691/691 [==============================] - 7s 10ms/step - loss: 0.2808 - accuracy: 0.9162 - val_loss: 0.2282 - val_accuracy: 0.932

In [86]:
net.build_inference_model()

In [90]:
net.encoder_model.summary()
combined_validation_data = list(zip(x_raw_val,y_raw_val))
np.random.shuffle(combined_validation_data)
(x_val,y_val) = zip(*combined_validation_data)
x_val_500,y_val_500 = np.array(x_val[:500]),np.array(y_val[:500])

Model: "model_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_10 (Embedding)     (None, None, 125)         3375      
_________________________________________________________________
lstm_40 (LSTM)               [(None, 256), (None, 256) 391168    
Total params: 394,543
Trainable params: 394,543
Non-trainable params: 0
_________________________________________________________________


In [92]:
net.evaluate(x_val_500,y_val_500)

patila  पतीला  पतिला
nakum  नकुम  नाकुम
punaravalokan  पुनरावलोकन  पुनरावलों
rajkaran  राजकरण  राजकरण
hitake  हाईटेक  हितके
shyamdas  श्यामदास  श्यामद्स
rasna  रसना  रसन्ना
celina  सेलीना  सेलिना
cricketeron  क्रिकेटरों  क्रिक्टरों
orlando  ओरलैंडो  ऑर्लेंडो
gangabai  गंगाबाई  गंगाबाई
thanks  थैंक्स  ठंक्स
najar  नजर  नज़र
duptta  दुपट्टा  दुप्ता
darvaajaa  दरवाजा  दरवाजा
typhoon  टाइफून  तिप्ण
marte  मारते  मारते
interpol  इंटरपोल  इंटरपोल
chhedte  छेड़ते  छेड़ते
sampradayikata  सांप्रदायिकता  संप्रदायिकता
hansda  हंसदा  हंसदा
kavval  कव्वाल  काववाल
mansikta  मानसिकता  मानीसिकता
dhanarthi  धनार्थी  धनरती
riyon  रियान  रियों
kanton  कांटों  कंतों
gomatinagar  गोमतीनगर  गोमेटनगर
startup  स्टार्टअप  स्टार्टप
bali  बली  बली
jockey  जॉकी  जॉकी
sampradayik  सांप्रदायिक  संप्रदायिक
ncg  एससीजी  एनएीजी
chitthiyon  चिठ्ठियों  चित्तियों
dhoondhte  ढूंढते  ढूंडते
profesion  प्रोफेशन  प्रोफ़ीशन
vipreet  विपरीत  विप्रेत
indy  इंडी  इंडी
baikunth  बैकुंठ  बेनुंता
deghat  देघाट  देघात
judge  जज  जुड

26.8

#Sweep configuration

In [20]:
sweep_config = {

    'method' : 'bayes',
    'metric' : {
        'name' : 'word_acc',
        'goal' : 'maximize'
    },
    'parameters': {
        'embedding_size': {
            'values': [16,32,64,128]
        },
        'num_encoder_layers': {
            'values': [1,2,3]
        },
        'num_decoder_layers': {
            'values': [1,2,3]
        },
        'hidden_layer_size': {
            'values': [32,64,128,256]
        },
        'cell_type' : {
            'values': ['LSTM','GRU','SimpleRNN']  
        },
        
        'drop_out_ratio': {
            'values': [0,0.2,0.4]
        },
        'batch_size': {
            'values': [64]
        },
        'epochs': {
            'values': [25]
        },
        'optimizer':{
            'values': ['adam','rmsprop']
        }
        
    }
}

In [17]:
def train():

    defaults = dict(
    embedding_size = 64,
    num_encoder_layers = 1,
    num_decoder_layers = 1,
    hidden_layer_size = 128,
    cell_type = "LSTM",
    drop_out_ratio = 0,
    batch_size = 64,
    epochs = 10,
    optimizer = 'rmsprop',
    )

    wandb.init(project="CS6910-Assignment3",config = defaults)
    config = wandb.config

    hyperparameters = {
    "embedding_size" : config.embedding_size,
    "num_encoder_layers" : config.num_encoder_layers,
    "num_decoder_layers" : config.num_decoder_layers,
    "hidden_layer_size" : config.hidden_layer_size,
    "cell_type" : config.cell_type,
    "drop_out_ratio": config.drop_out_ratio,
    "in_char_size": len(english_alpha2index),
    "out_char_size": len(hindi_alpha2index),
    "input_len": 25,
    }

    net = RNN(**hyperparameters)
    net.compile(optimizer = config.optimizer)
    net.fit(integer_encoded_x_train, decoder_input_data,decoder_output_data,batch_size = config.batch_size,epochs = config.epochs)
    net.build_inference_model()
    combined_validation_data = list(zip(x_raw_val,y_raw_val))
    np.random.shuffle(combined_validation_data)
    (x_val,y_val) = zip(*combined_validation_data)
    x_val_200,y_val_200 = np.array(x_val[:200]),np.array(y_val[:200])
    word_accuracy = net.evalute(x_val_200,y_val_200)
    wandb.log(({"word_acc": word_accuracy}))


In [21]:
sweep_id = wandb.sweep(sweep_config, entity="theindianwriter", project="CS6910-Assignment3")

Create sweep with ID: fre81alj
Sweep URL: https://wandb.ai/theindianwriter/CS6910-Assignment3/sweeps/fre81alj


In [30]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: d7hah238 with config:
wandb: 	batch_size: 64
wandb: 	cell_type: SimpleRNN
wandb: 	drop_out_ratio: 0
wandb: 	embedding_size: 16
wandb: 	epochs: 25
wandb: 	hidden_layer_size: 128
wandb: 	num_decoder_layers: 2
wandb: 	num_encoder_layers: 1
wandb: 	optimizer: adam
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/25
622/622 [==============================] - 33s 50ms/step - loss: 1.1872 - accuracy: 0.7342 - val_loss: 0.8955 - val_accuracy: 0.7658
Epoch 2/25
622/622 [==============================] - 31s 50ms/step - loss: 0.7961 - accuracy: 0.7850 - val_loss: 0.9001 - val_accuracy: 0.7670
Epoch 3/25
622/622 [==============================] - 30s 49ms/step - loss: 0.7715 - accuracy: 0.7905 - val_loss: 1.2968 - val_accuracy: 0.7639
Epoch 4/25
622/622 [==============================] - 30s 48ms/step - loss: 0.7755 - accuracy: 0.7904 - val_loss: 0.9238 - val_accuracy: 0.7684
Epoch 5/25
622/622 [==============================] - 31s 49ms/step - loss: 0.7356 - accuracy: 0.7962 - val_loss: 0.9239 - val_accuracy: 0.7696
Epoch 6/25
622/622 [==============================] - 31s 49ms/step - loss: 0.7277 - accuracy: 0.7976 - val_loss: 0.9220 - val_accuracy: 0.7698
Epoch 7/25
622/622 [==============================] - 30s 49ms/step - loss: 0.7202 - accuracy: 0.8002 - val_loss: 0.9419 - val_accuracy:

epoch,24
loss,0.63758
accuracy,0.77967
val_loss,1.09075
val_accuracy,0.72267
_runtime,768
_timestamp,1621690588
_step,24
best_val_loss,0.89554
best_epoch,0


epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
accuracy,▁▄▅▅▅▆▆▆▆▇▇▇▇█▇▇███▄▃▃▃▃▃
val_loss,▁▁█▁▁▁▂▃▂▃▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄
val_accuracy,▇▇▇▇▇▇█▇███▇██▇▇███▂▂▂▂▂▁
_runtime,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▇▇▇▇██
_timestamp,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▇▇▇▇██
_step,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██


wandb: Agent Starting Run: dg7f27lc with config:
wandb: 	batch_size: 32
wandb: 	cell_type: GRU
wandb: 	drop_out_ratio: 0.2
wandb: 	embedding_size: 128
wandb: 	epochs: 25
wandb: 	hidden_layer_size: 128
wandb: 	num_decoder_layers: 2
wandb: 	num_encoder_layers: 4
wandb: 	optimizer: adam
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/25
 694/1244 [===============>..............] - ETA: 8s - loss: 1.2912 - accuracy: 0.7358

wandb: Ctrl + C detected. Stopping sweep.


#A single runnable Model

In [ ]:
batch_size = 64  
epochs = 25
latent_dim = 256

In [ ]:
embedding_inputs = keras.Input(shape=(25))
embedding_layer = Embedding(len(english_alpha2index), 125 , input_length=25)
encoder_inputs = embedding_layer(embedding_inputs)
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, len(hindi_alpha2index)))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(len(hindi_alpha2index), activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([embedding_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [integer_encoded_x_train, decoder_input_data],
    decoder_output_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.1,
)

Epoch 1/25
622/622 [==============================] - 191s 302ms/step - loss: 1.1451 - accuracy: 0.7386 - val_loss: 0.9244 - val_accuracy: 0.7665
Epoch 2/25
622/622 [==============================] - 201s 323ms/step - loss: 0.7544 - accuracy: 0.8006 - val_loss: 0.8717 - val_accuracy: 0.7954
Epoch 3/25
622/622 [==============================] - 185s 297ms/step - loss: 0.6064 - accuracy: 0.8345 - val_loss: 0.7435 - val_accuracy: 0.8184
Epoch 4/25
622/622 [==============================] - 184s 295ms/step - loss: 0.4774 - accuracy: 0.8660 - val_loss: 0.6884 - val_accuracy: 0.8375
Epoch 5/25
622/622 [==============================] - 184s 295ms/step - loss: 0.3836 - accuracy: 0.8901 - val_loss: 0.6404 - val_accuracy: 0.8472
Epoch 6/25
622/622 [==============================] - 184s 295ms/step - loss: 0.3134 - accuracy: 0.9090 - val_loss: 0.5655 - val_accuracy: 0.8677
Epoch 7/25
622/622 [==============================] - 184s 296ms/step - loss: 0.2615 - accuracy: 0.9227 - val_loss: 0.5317 -

In [ ]:
model.save("/content/drive/MyDrive/s2s")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/s2s/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/s2s/assets


In [ ]:
model = keras.models.load_model("/content/drive/MyDrive/s2s")

In [ ]:
#model.summary()

embedding_inputs = model.input[0]  # input_1
embedding_l1 = model.layers[1]
encoder_inputs = embedding_l1(embedding_inputs)
encoder_lstm = model.layers[3]
encoder_outputs, state_h_enc, state_c_enc = encoder_lstm(encoder_inputs)
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(embedding_inputs, encoder_states)
encoder_model.summary()

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_5")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[4]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[5]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)
decoder_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 25)]              0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 25, 125)           3375      
_________________________________________________________________
lstm_10 (LSTM)               [(None, 256), (None, 256) 391168    
Total params: 394,543
Trainable params: 394,543
Non-trainable params: 0
_________________________________________________________________
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, None, 131)]  0                                            
_____________________________________________________________________

In [ ]:
def decode_sequence(word):
    input_seq = get_integer_encode(word,english_alpha2index)
    input_seq = input_seq.reshape(1,25)
    
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1, len(hindi_alpha2index)))
    # Populate the first character of target sequence with the start character.
    target_seq[0,0, hindi_alpha2index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_word = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])

        sampled_char = hindi_index2alpha[sampled_token_index]
        

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_word) > 25:
            stop_condition = True
            break

        decoded_word += sampled_char

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1,len(hindi_alpha2index)))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_word

In [ ]:
def test():
    correct = 0
    for english_word,hindi_word in zip(x_raw_test,y_raw_test):
    
        predicted_hindi_word = decode_sequence(english_word)
        print(english_word+"  "+hindi_word+"  "+predicted_hindi_word)
        if predicted_hindi_word == hindi_word:
            correct += 1

    acc = (correct/x_raw_test.shape[0])*100
    return acc
    